#### Install Pypop
https://github.com/alexlancaster/pypop?tab=readme-ov-file#readme

#### Install Arlequin
http://cmpg.unibe.ch/software/arlequin3/

## ARLEQUIN

In [ ]:
'''
Input preparation for the HW, LD and haplotypic frequency estimation in the Arlequin Software
'''


'''
txt_file example
id    A        A       B      B        C       C      DRB1    DRB1   DQB1    DQB1    DPB1    DPB1
1   24:02   24:02   49:01   51:01   06:02   15:02   04:03   14:54   03:02   05:03   04:01   04:01
2   24:02   68:01   18:01   35:03   04:01   07:01   11:04   12:01   03:01   03:01   02:01   04:02
3   02:01   11:01   44:05   51:01   02:02   15:02   16:01   16:01   05:02   05:02   04:02   10:01
4   02:17   32:01   15:01   51:01   03:04   15:02   01:01   14:54   05:01   05:03   04:01   104:01
5   02:01   24:02   18:01   35:03   04:01   12:03   11:04   11:04   03:01   03:01   04:01   14:01
6   02:01   11:01   15:01   39:01   04:01   12:03   12:01   16:01   03:01   05:02   04:01   15:01
7   01:01   23:01   07:05   49:01   07:01   15:05   07:01   11:01   02:02   03:01   02:01   04:01
8   02:01   11:01   35:01   51:01   04:01   15:02   01:01   13:01   05:01   06:03   02:01   03:01
'''

def arlequin_inputs(txt_file, #txt file
                    hla_gene=None, #A or B or C...
                    all_genes=None, #A,B,C,DRB1,DQB1,DPB1
                    include_missing_data=None, #TRUE: if you have missing data
                    num_of_genes=None, #num_of_genes
                    HW=None, #=1 if you want an arlequin input file for Hardy-Weinberg equilibrium
                    LD=None, #=1 if you want an arlequin input file for Linkage Disequillibrium
                    HF=None, #=1 if you want an arlequin input file for Haplotypic Frequencies
                    path=None #just the path of the txt file
                   ):    
    '''
    txt_file should contain the genotypes with unknown phasing in any resolution of 
    the HLA genes A, B, C, DRB1, DQB1, DPB1 in this order only.
    If there are not data about some of these genes, just put None values on the respective columns.
    In this case set include_missing_data=True.
    
    Your 'path' directory should contain 3 subdirectories: HW/, LD/ and HF/.
    '''
    
    if include_missing_data and txt_file == 'mylopotamos.txt':
        raise Exception('This file has no missing data.')
 
    if hla_gene and all_genes or hla_gene and num_of_genes or num_of_genes and not all_genes:
        raise Exception('You can pick only 1 gene, or all of them, or the first n of them.')
    
    if num_of_genes and not 1<num_of_genes<6 and not isinstance(num_of_genes, int):
        raise Exception("num_of_genes should be an integer from 2 to 5 indicating the first n genes that you wish to be in the input for Arlequin")
    
    if hla_gene and HF or hla_gene and LD:
        raise Exception('You cannot create haplotypes or calculate Linkage Disequillibrium with just 1 genetic loci.')
    
    if HW and LD or HW and HF or LD and HF:
        raise Exception('You can choose only 1 type of analysis for the input preparation.')
    
    with open(txt_file, 'r') as pop:
        pop = pop.readlines()

    pop = [x.strip('\n').split('\t') for x in pop]
    
    if HF:
        pop = [[x.replace(' ','') for x in y] for y in pop]
    else:
        pop = [[x.replace(' ','').replace(':','') for x in y] for y in pop]
    
    if not include_missing_data:
        pop_tmp = [x for x in pop if sum('' == s for s in x)==0]
    else:
        pop_tmp = pop.copy()
    
    if all_genes:
        
        pop = [['A'+x[1],'A'+x[2],'B'+x[3],'B'+x[4],'C'+x[5],'C'+x[6],'DR'+x[7],'DR'+x[8],'DQ'+x[9],'DQ'+x[10],'DP'+x[11],'DP'+x[12]] for x in pop_tmp[1:]]
        
        if num_of_genes:
            pop = [x[:2*num_of_genes] for x in pop]
        
        obs_counts_pop_A = dict(Counter(map(tuple, pop))) 
        obs_counts_pop_A = {k:v for k,v in sorted(obs_counts_pop_A.items(), key = lambda item:item[1], reverse=True)}
        
        if num_of_genes:
            
            if num_of_genes == 2:
                input_tmp = ['\t'+str(v)+'\t'+k[0]+'\t'+k[2]+'\n'+'\t\t'+k[1]+'\t'+k[3]+'\n' for k,v in obs_counts_pop_A.items()]

            if num_of_genes == 3:
                input_tmp = ['\t'+str(v)+'\t'+k[0]+'\t'+k[2]+'\t'+k[4]+'\n'+'\t\t'+k[1]+'\t'+k[3]+'\t'+k[5]+'\n' for k,v in obs_counts_pop_A.items()]

            if num_of_genes == 4:
                input_tmp = ['\t'+str(v)+'\t'+k[0]+'\t'+k[2]+'\t'+k[4]+'\t'+k[6]+'\n'+'\t\t'+k[1]+'\t'+k[3]+'\t'+k[5]+'\t'+k[7]+'\n' for k,v in obs_counts_pop_A.items()]

            if num_of_genes == 5:
                input_tmp = ['\t'+str(v)+'\t'+k[0]+'\t'+k[2]+'\t'+k[4]+'\t'+k[6]+'\t'+k[8]+'\n'+'\t\t'+k[1]+'\t'+k[3]+'\t'+k[5]+'\t'+k[7]+'\t'+k[9]+'\n' for k,v in obs_counts_pop_A.items()]
        
        else:
            
            input_tmp = ['\t'+str(v)+'\t'+k[0]+'\t'+k[2]+'\t'+k[4]+'\t'+k[6]+'\t'+k[8]+'\t'+k[10]+'\n'+'\t\t'+k[1]+'\t'+k[3]+'\t'+k[5]+'\t'+k[7]+'\t'+k[9]+'\t'+k[11]+'\n' for k,v in obs_counts_pop_A.items()]
            
        input_tmp = [str(input_tmp.index(x)+1)+x for x in input_tmp ]
        
        if include_missing_data:
            input_tmp = [x.replace('\tA\n', '\t?\n').replace('\tB\n', '\t?\n').replace('\tC\n', '\t?\n').replace('\tDR\n', '\t?\n').replace('\tDQ\n', '\t?\n').replace('\tDP\n', '\t?\n').replace('\tA\t', '\t?\t').replace('\tB\t', '\t?\t').replace('\tC\t', '\t?\t').replace('\tDR\t', '\t?\t').replace('\tDQ\t', '\t?\t') for x in input_tmp]       
        
        to_write = ''.join(input_tmp)
        
        
    if not all_genes:
        
        if hla_gene in ('DRB1','DQB1','DPB1'):
            pop = [[hla_gene[0:2]+x.replace(':','') for x in y] for y in pop_tmp]
        else:
            pop = [[hla_gene+x.replace(':','') for x in y] for y in pop_tmp]

        if hla_gene == 'A':
            pop = [x[1:3] for x in pop[1:]]
        elif hla_gene == 'B':
            pop = [x[3:5] for x in pop[1:]]
        elif hla_gene == 'C':
            pop = [x[5:7] for x in pop[1:]]
        elif hla_gene == 'DRB1':
            pop = [x[7:9] for x in pop[1:]]
        elif hla_gene == 'DQB1':
            pop = [x[9:11] for x in pop[1:]]
        elif hla_gene == 'DPB1':
            pop = [x[11:] for x in pop[1:]] 

        pop = sorted(pop, key = lambda x:(x[0],x[1]))

        obs_counts_pop_A = dict(Counter(map(tuple, pop))) 

        input_tmp = ['\t'+str(v)+'\t'+k[0]+'\n'+'\t\t'+k[1]+'\n' for k,v in obs_counts_pop_A.items()]

        input_tmp = [str(input_tmp.index(x)+1)+x for x in input_tmp ]
           
        if include_missing_data:
            input_tmp = [x.replace('\tA\n', '\t?\n').replace('\tB\n', '\t?\n').replace('\tC\n', '\t?\n').replace('\tDR\n', '\t?\n').replace('\tDQ\n', '\t?\n').replace('\tDP\n', '\t?\n') for x in input_tmp]       
        
        to_write = ''.join(input_tmp)  

    
    place = txt_file.split('.')[0]
        
    if HW:
        path = path + 'HW/'
    elif LD:
        path = path + 'LD/'
    elif HF:
        path = path + 'HF/'
    else:
        raise Exception('Something went wrong. Check the HW, LD and HF parameters.')
    
    n1 ='{'
    n2='}'
    
    if all_genes:
        if not num_of_genes:
            n = 6
        else:
            n = num_of_genes
    
    if HW:
        title = f'Hardy-Weinberg equilibrium exact test for {len(pop)} individuals from {place}'
    elif LD:
        title = f'Linkage Disequillibrium test for {n} loci of {len(pop)} individuals from {place}'
    elif HF:
        title = f'Haplotypic Frequencies estimation for {n} loci of {len(pop)} individuals from {place}'
    else:
        raise Exception('Something went wrong. Check the HW, LD and HF parameters.')
    
    final_input = f'''[Profile]

     Title="{title}"
     NbSamples=1
     DataType=STANDARD
     GenotypicData=1
     LocusSeparator=TAB
     GameticPhase=0
     RecessiveData=0
     MissingData="?"

[Data] 

[[Samples]] 

    SampleName="{len(pop)} from {place}"
    SampleSize={len(pop)} # Number of diploid individuals in sample 
    SampleData= {n1}

{to_write}{n2}'''
    
    if include_missing_data:
        md = '_missing_data'
    else:
        md = ''
    
    if all_genes:
        
        if HW:
            
            if not num_of_genes:
                if os.path.exists(f'{path}{place}_Arlequin_HW_all{md}.arp'):
                    os.remove(f'{path}{place}_Arlequin_HW_all{md}.arp')
                    
                with open(f'{path}{place}_Arlequin_HW_all{md}.arp', 'w') as h:
                    h.write(final_input)
            else:
                
                if os.path.exists(f'{path}{place}_Arlequin_HW_{n}{md}.arp'):
                    os.remove(f'{path}{place}_Arlequin_HW_{n}{md}.arp')

                with open(f'{path}{place}_Arlequin_HW_{n}{md}.arp', 'w') as h:
                    h.write(final_input)
                    
        if LD:
            
            if not num_of_genes:
                if os.path.exists(f'{path}{place}_Arlequin_LD_all{md}.arp'):
                    os.remove(f'{path}{place}_Arlequin_LD_all{md}.arp')

                with open(f'{path}{place}_Arlequin_LD_all{md}.arp', 'w') as h:
                    h.write(final_input)
            else:
                
                if os.path.exists(f'{path}{place}_Arlequin_LD_{n}{md}.arp'):
                    os.remove(f'{path}{place}_Arlequin_LD_{n}{md}.arp')

                with open(f'{path}{place}_Arlequin_LD_{n}{md}.arp', 'w') as h:
                    h.write(final_input)
                    
        if HF:
            
            if os.path.exists(f'{path}{place}_Arlequin_HF_{n}{md}.arp'):
                os.remove(f'{path}{place}_Arlequin_HF_{n}{md}.arp')

            with open(f'{path}{place}_Arlequin_HF_{n}{md}.arp', 'w') as h:
                h.write(final_input)
                    
    if not all_genes:        
        
        if HW:
            
            if os.path.exists(f'{path}{place}_Arlequin_HW_{hla_gene}{md}.arp'):
                os.remove(f'{path}{place}_Arlequin_HW_{hla_gene}{md}.arp')

            with open(f'{path}{place}_Arlequin_HW_{hla_gene}{md}.arp', 'w') as h:
                h.write(final_input)
                
        if LD or HF:
            raise Exception('Something went wrong with the analysis and the number of the genes.')

# arlequin_inputs('Crete_75.txt', hla_gene=None, all_genes=True, include_missing_data=False, num_of_genes=None, HW=True, LD=None, HF=None, path = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/')


In [ ]:
'''
This function merges multiple .arp files (Arlequin v.3.5.2.2 format) for the SAME analysis, 
so be careful when choosing the inputs. 
'''
def merge_arps(list_with_arps, path, remove_inputs=None):
    
    if len(list_with_arps) < 2:
        raise Exception('The list should contain at least 2 .arp files for merging.')
    
    os.chdir(path)
        
    for x in range(len(list_with_arps)):
        if not os.path.exists(list_with_arps[x]):
            raise Exception(f'The file {list_with_arps[x]} does not exist. Choose it from the correct folder.')
        
    pop_num = len(list_with_arps)
    
    to_write_list = []
    
    for x in range(len(list_with_arps)):
        with open(list_with_arps[x], 'r') as tmp:
            tmp = tmp.readlines()
            
        tmp = [x.strip('\n') for x in tmp]    
        tmp = [tmp[x+1:] for x in range(len(tmp)) if '[[Samples]]' in tmp[x]]        
        
        to_write_list.append(tmp[0])

    header = [[f'''[Profile]

     Title="Genotypes of all populations together"
     NbSamples={pop_num}
     DataType=STANDARD
     GenotypicData=1
     LocusSeparator=TAB
     GameticPhase=0
     RecessiveData=0
     MissingData="?"

[Data] 

[[Samples]]''']]
    
    final_output = header + to_write_list
    
    if os.path.exists(f'{path}merged.arp'):
        os.remove(f'{path}merged.arp')

    with open(f'{path}merged.arp', 'w') as h:
        for _list in final_output:
            for _string in _list:
                h.write(str(_string) + '\n')   
                
    if remove_inputs:
        for x in range(len(list_with_arps)):
            if os.path.exists(list_with_arps[x]):
                os.remove(list_with_arps[x])
# merge_arps(['/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/Chania.arp', '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/Rethymno.arp', '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/Heraklion.arp', '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/Lasithi.arp'], path = '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/')


In [ ]:
    
'''
The order of the populations should be Mylopotamos, rest, Crete and the analysis
should contain 6 HLA genes with the order A, B, C, DRB1, DQB1, DPB1.

The p-values and the haplotypic frequencies cannot be obtained from .xml files 
that have come from different types of analyses, or from different .ars files 
or from different parameters in the .arp files or from different types of inputs in 
general than the ones we used. Our data were genotype data with unknown phasing.

The .xml file must contain only once in its name the expression 
_{analysis}_{number_of_loci}_ , where {analysis} can either be 'HF','HW' or 'LD'
and {number_of_loci} can either be 2,3,4,5 or 6. Examples of this  expression: 
_HF_6_ , _LD_5_ , _HF_4_ .
'''

def obtain_pvalue_from_Arlequin_xml(xml_file, analysis):    
    if not os.path.exists(xml_file):
        raise Exception('This file does not exist. Chose the correct one from the correct folder.')
    
    if analysis not in ('HW', 'LD', 'HF'):
        raise Exception("analysis can only be 'HW', 'LD' or 'HF'")
    
    if xml_file.split(f'{analysis}_')[1][0] == 'a':
        n = 6
    else:
        n = int(xml_file.split(f'{analysis}_')[1][0])
    
    with open(xml_file, 'r') as h:
        h = h.readlines()
        
    h = [x.strip('\n') for x in h]              
        
    if analysis == 'HF': 
        
        start = [i for i, x in enumerate(h) if x ==  '    #   Haplotype     Freq.      s.d.']
        end = [i for i, x in enumerate(h) if 'Sum of all' in x]
        
        if len(start) != len(end):
            raise Exception('Something went wrong while parsing the HF.xml file. Could not retrieve the frequencies. Check the .xml file if there is something unusual happening between the populations.')
        
        haplotypes = [h[start[x]+3:end[x]-2] for x in range(len(start))]
        haplotypes = [[y.strip(' ').split(' ') for y in x] for x in haplotypes]
        haplotypes = [[z.split('\t') for y in x for z in y if z] for x in haplotypes]
        
        haplotypes_tmp = []
        
        for x in range(len(haplotypes)):
            haplotypes_tmp.append(list(itertools.chain.from_iterable(haplotypes[x])))
        
        haplotypes = [[haplotypes_tmp[x][y:y+n] for y in list(range(4,len(haplotypes_tmp[x]),n+5))] for x in range(len(haplotypes_tmp))]
        
        hfs = [[haplotypes_tmp[x][y-2] for y in list(range(4,len(haplotypes_tmp[x]),n+5))]  for x in range(len(haplotypes_tmp))]  
         
        for x in range(len(haplotypes)):
            for y in range(len(haplotypes[x])):
                haplotypes[x][y].append(hfs[x][y])
       
        haplotypes = [[['~'.join(haplotypes[x][y][0:n]).replace('A','A*').replace('B','B*').replace('C','C*').replace('DR','DRB1*').replace('DQ','DQB1*').replace('DP','DPB1*'),haplotypes[x][y][n]] for y in list(range(len(haplotypes[x])))] for x in range(len(haplotypes))]
        
        hf_list = []
        
        for x in range(len(haplotypes)):
            hf_dict = {}
            
            for y in list(range(len(haplotypes[x]))):                
                hf_dict[haplotypes[x][y][0]]=float(haplotypes[x][y][1])             
            
            hf_dict = {k: v for k, v in sorted(hf_dict.items(), key=lambda item: item[1], reverse=True)}
            hf_list.append(hf_dict.copy())
        
        return hf_list
    
    elif analysis == 'LD':
        
        combos = len(list(itertools.combinations(list(range(n)), 2)))
         
        pop_num = len([h[x] for x in range(len(h)) if 'Test of linkage disequilibrium for all pairs of loci:' == h[x] ])
        
        h = [h[x+8:x+8+3*combos] for x in range(len(h)) if 'Test of linkage disequilibrium for all pairs of loci:' == h[x]]
        ### the 3 in 3*combos stands for the number of lines in the .xml output LD file and not in the pop_num
        
        ld_pvals = [[h[x][y],float(re.findall(r'\(P = .*?,', h[x][y+2])[0].split('=  ')[1].strip(','))] for y in list(range(0,3*combos,3)) for x in range(len(h))]
        
        pops_pvals = []
        
        for x in list(range(pop_num)):
            tmp = []
            
            for y in range(x,len(ld_pvals),pop_num):
                tmp.append(ld_pvals[y][1])
            
            pops_pvals.append(tmp)     
        
        return pops_pvals
    
    elif analysis == 'HW':
        
        pop_num = len([h[x] for x in range(len(h)) if "Locus  #Genot       Obs.Het.     Exp.Het.   P-value     s.d.  Steps done" == h[x] ])
        
        h = [h[x+2:x+2+n] for x in range(len(h)) if "Locus  #Genot       Obs.Het.     Exp.Het.   P-value     s.d.  Steps done" == h[x]]
        h = [[x.strip(' ').split(' ') for x in y] for y in h]
        h = [[z for y in x for z in y if z] for x in h]

        pvals = [float(h[x][y]) for y in list(range(4, 4+7*(n-1)+1, 7)) for x in range(len(h))]
        
        pops_pvals = []
        
        for x in list(range(pop_num)):
            tmp = []
            
            for y in range(x,len(pvals),pop_num):
                tmp.append(pvals[y])
            
            pops_pvals.append(tmp)     
        
        return pops_pvals
    
DKMS_HW_path = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/HW/Crete_75_Arlequin_HW_all.res/'
DKMS_LD_path = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/LD/Crete_75_Arlequin_LD_all.res/'
DKMS_HF_path_6 = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/HF/Crete_75_Arlequin_HF_6.res/'

# arlequin_DKMS_HW = obtain_pvalue_from_Arlequin_xml(f'{DKMS_HW_path}Crete_75_Arlequin_HW_all.xml', 'HW')
# arlequin_DKMS_LD = obtain_pvalue_from_Arlequin_xml(f'{DKMS_LD_path}Crete_75_Arlequin_LD_all.xml', 'LD')
# arlequin_DKMS_HF = obtain_pvalue_from_Arlequin_xml(f'{DKMS_HF_path_6}Crete_75_Arlequin_HF_6.xml','HF')

In [ ]:
### Run Arlequin 1000 times and get the mean of HWE p-values of Crete

a = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/HW/'
b = '../../../../arlecore_linux/arlecore3522_64bit HWequil.ars Crete_75_Arlequin_HW_all.arp'
c = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/HW/Crete_75_Arlequin_HW_all.res/'
d = 'mv Crete_75_Arlequin_HW_all.xml Crete_75_Arlequin_HW_6.xml'
e = '/home/manos/Programs/Arlequin/cretan_HLA/DKMS/1835/HW/Crete_75_Arlequin_HW_all.res/Crete_75_Arlequin_HW_6.xml'

def run_Arlequin_HWE(runs):
        
    pvals = []
    
    for run in range(runs):
        print(run)
        os.chdir(a)
        os.system(b)
        os.chdir(c)
        os.system(d)
        
        tmp_pvals = obtain_pvalue_from_Arlequin_xml(e, 'HW')
        pvals.append(tmp_pvals[0])
        
    os.chdir('/home/manos/Desktop/Manos/MSc-Bioinformatics/Thesis/scripts/Cretan_project/')
    
    return pvals

def mean_confidence_interval(data, confidence=0.95):    
    
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

arlequin_DKMS_HW = run_Arlequin_HWE(1000)

m_Crete_A, ci1_Crete_A, ci2_Crete_A = mean_confidence_interval([x[0] for x in arlequin_DKMS_HW], confidence=0.95)
m_Crete_B, ci1_Crete_B, ci2_Crete_B = mean_confidence_interval([x[1] for x in arlequin_DKMS_HW], confidence=0.95)
m_Crete_C, ci1_Crete_C, ci2_Crete_C = mean_confidence_interval([x[2] for x in arlequin_DKMS_HW], confidence=0.95)
m_Crete_DRB1, ci1_Crete_DRB1, ci2_Crete_DRB1 = mean_confidence_interval([x[3] for x in arlequin_DKMS_HW], confidence=0.95)
m_Crete_DQB1, ci1_Crete_DQB1, ci2_Crete_DQB1 = mean_confidence_interval([x[4] for x in arlequin_DKMS_HW], confidence=0.95)
m_Crete_DPB1, ci1_Crete_DPB1, ci2_Crete_DPB1 = mean_confidence_interval([x[5] for x in arlequin_DKMS_HW], confidence=0.95)

Crete_HW = [m_Crete_A,m_Crete_B,m_Crete_C,m_Crete_DRB1,m_Crete_DQB1,m_Crete_DPB1]

In [ ]:
def Arlequin_HWE_pvalues(arlequin_DKMS,log10=None):
    
    if log10:
        
        # We are adding 0.0001 on 0.0 p-values, because log tranformation is not posssible        
        arlequin_DKMS = [-np.log10(x) for x in [0.0001 if x==0 else x for x in arlequin_DKMS]]

    fig, ax = plt.subplots()
    
    plt.scatter(list(range(1,31,5)), arlequin_DKMS, marker='*', color = 'blue',label=f'{n_cretans} Cretans',s=35)
    
    if log10:
        plt.plot([0.8, 28.2],[-np.log10(0.05/6), -np.log10(0.05/6)], color ='red', label='a = -log10(0.05/6)')
        ax.legend()
#         ax.legend(loc='upper left')
        plt.ylabel('-log10 transformed p-values', fontsize = 13)
    else:
        plt.plot([0.8, 28.2],[0.05/6, 0.05/6], color ='red', label='a = 0.05/6')
    ### Corrected threshold: a / (number of genetic loci) where a == 0.05 and (number of genetic loci) == 6
        ax.legend(loc='upper right')
        plt.ylabel('p-values', fontsize = 15)
    
    plt.xticks(list(range(1,31,5)), ['A', 'B', 'C', 'DRB1', 'DQB1', 'DPB1'], fontsize=13) 
    plt.title('HWE p-values (mean out of\n1000 runs) on 6 HLA loci', fontsize=15)       
    
#     fig.tight_layout()
#     fig.set_dpi(300)    
    
#     if log10:
#         plt.savefig("HWE p-values log transformation.png")   
#     else:
#         plt.savefig("HWE p-values.png")   
       
    plt.show()

# Arlequin_HWE_pvalues(Crete_HW)
Arlequin_HWE_pvalues(Crete_HW, log10=True)

In [ ]:
def Arlequin_LD_pvalues(arlequin_DKMS,log10=None):
    
    if log10:
        
        # We are adding 0.0001 on 0.0 p-values, because log tranformation is not posssible
        arlequin_DKMS = [-np.log10(x) for x in [0.0001 if x==0 else x for x in arlequin_DKMS]]
    
    fig, ax = plt.subplots()
    
    plt.scatter(list(range(1,76,5)), arlequin_DKMS, marker='*', color = 'blue',label='1835 Cretans',s=35)
    
    if log10:
        plt.plot([0.8, 73.2],[-np.log10(0.05/15), -np.log10(0.05/15)], color ='red', label='a = -log10(0.05/15)')
        plt.ylabel('-log10 transformed p-values', fontsize = 12)
        ax.legend(loc='lower left')
    else:
        plt.plot([0.8, 73.2],[0.05/15, 0.05/15], color ='red', label='a = 0.05/15')
    ### Corrected threshold: a / (number of loci pairs) where a == 0.05 and (number of loci pairs) == 15
        plt.ylabel('p-values', fontsize = 15)    
        ax.legend(loc='upper left')
    
    
    plt.xticks(list(range(1,76,5)), ['A-B','A-C','B-C','A-DRB1','B-DRB1','C-DRB1','A-DQB1','B-DQB1','C-DQB1','DRB1-DQB1','A-DPB1','B-DPB1','C-DPB1','DRB1-DPB1','DQB1-DPB1'], fontsize=10, rotation=-90)
    plt.title("Comparison of LD p-values per loci pair", fontsize=15)       
    
#     fig.tight_layout()    
#     fig.set_dpi(300)
    
#     if log10:
#         plt.savefig("LD p-values log transformation.png")
#     else:
#         plt.savefig("LD p-values.png") 
       
    plt.show()

arlequin_DKMS_LD = obtain_pvalue_from_Arlequin_xml(f'{DKMS_LD_path}Crete_75_Arlequin_LD_all.xml', 'LD')    

# Arlequin_LD_pvalues(arlequin_DKMS_LD[0],log10=None)
Arlequin_LD_pvalues(arlequin_DKMS_LD[0],log10=True)

In [ ]:
### Run Arlequin 1000 times and get the mean of HWE p-values of all Prefectures 

a = '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/HW/'
b = '../../../../arlecore_linux/arlecore3522_64bit HWequil.ars merged.arp'
c = '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/HW/merged.res/'
d = 'mv merged.xml merged_HW_6.xml'
e = '/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/HW/merged.res/merged_HW_6.xml'

def run_Arlequin_HWE_Prefectures(runs):
    
    pvals_Ch = []
    pvals_Ret = []
    pvals_Her = []
    pvals_Las = []
    
    for run in range(runs):
        print(run)
        os.chdir(a)
        os.system(b)
        os.chdir(c)
        os.system(d)

        tmp_pvals = obtain_pvalue_from_Arlequin_xml(e, 'HW')

        pvals_Ch.append(tmp_pvals[0])
        pvals_Ret.append(tmp_pvals[1])
        pvals_Her.append(tmp_pvals[2])
        pvals_Las.append(tmp_pvals[3])
    
    os.chdir('/home/manos/Desktop/Manos/MSc-Bioinformatics/Thesis/scripts/Cretan_project/')
    
    return pvals_Ch, pvals_Ret, pvals_Her, pvals_Las

pvals_Ch, pvals_Ret, pvals_Her, pvals_Las = run_Arlequin_HWE_Prefectures(1000)

In [ ]:
### HWE AND LD COMPARISON PER PREFECTURE

def prefectures_HWE_LD_pvalues(pvals, analysis, log10=None):
    
    if analysis not in ('HW', 'LD'):
        raise Exception('analysis can only be HW or LD.')
       
    if log10:
    
        # We are adding 0.0001 on 0.0 p-values, because log tranformation is not posssible        
        pvals = [-np.log10(z) for z in [[0.0001 if y==0 else y for y in x] for x in pvals]]

    fig, ax = plt.subplots()
    
    plt.scatter(list(range(1,len(pvals[0])+1)), pvals[0], marker='o',color='blue',label='Chania (455)',s=35,alpha=0.5)
    plt.scatter(list(range(1,len(pvals[1])+1)), pvals[1], marker='*',color='magenta',label='Rethymno (280)',s=35,alpha=0.5)
    plt.scatter(list(range(1,len(pvals[2])+1)), pvals[2], marker='>',color='green',label='Heraklion (678)',s=35,alpha=0.5)
    plt.scatter(list(range(1,len(pvals[3])+1)), pvals[3], marker='s',color='orange',label='Lasithi (331)',s=35,alpha=0.5)
    
    if log10:
        
        if analysis == 'HW':
            plt.plot([0.8, len(pvals[0])+0.2],[-np.log10(0.05/6), -np.log10(0.05/6)], color ='red', label='a = -log10(0.05/6)')
        elif analysis == 'LD':
            plt.plot([0.8, len(pvals[0])+0.2],[-np.log10(0.05/15), -np.log10(0.05/15)], color ='red', label='a = -log10(0.05/15)')
    
        plt.ylabel('-log10 transformed p-values', fontsize = 10)
        
    else:
        
        if analysis == 'HW':
            plt.plot([0.8, len(pvals[0])+0.2],[0.05/6, 0.05/6], color ='red', label='a = 0.05/6')
        elif analysis == 'LD':
            plt.plot([0.8, len(pvals[0])+0.2],[0.05/15, 0.05/15], color ='red', label='a = 0.05/15')
    ### Corrected threshold: a / (number of genetic loci/pairs) where a == 0.05 and (number of genetic loci) == 6/15
        plt.ylabel('p-values', fontsize = 15)    
        
    ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')    
    
    if analysis =='HW':
        plt.xticks(list(range(1,len(pvals[0])+1)), ['A', 'B', 'C', 'DRB1', 'DQB1', 'DPB1'], fontsize=13) 
        plt.title('HWE p-values (mean out of 1000 runs)\non 6 HLA loci per Prefecture', fontsize=15)    
    elif analysis == 'LD':
        plt.xticks(list(range(1,len(pvals[0])+1)), ['A-B','A-C','B-C','A-DRB1','B-DRB1','C-DRB1','A-DQB1','B-DQB1','C-DQB1','DRB1-DQB1','A-DPB1','B-DPB1','C-DPB1','DRB1-DPB1','DQB1-DPB1'], fontsize=10, rotation=-90)
        plt.title("LD p-values per loci pair\non 6 HLA loci per Prefecture", fontsize=15)
    
#     fig.tight_layout()
#     fig.set_dpi(300)    
    
#     if log10:
        
#         if analysis == 'HW':
#             plt.savefig('Prefectures_Analysis/Prefecture HWE p-values log transformation.png')
#         elif analysis == 'LD':
#             plt.savefig('Prefectures_Analysis/Prefecture LD p-values log transformation.png')
#     else:
        
#         if analysis == 'HW':
#             plt.savefig('Prefectures_Analysis/Prefecture HWE p-values.png')   
#         elif analysis == 'LD':
#             plt.savefig('Prefectures_Analysis/Prefecture LD p-values.png')
       
    plt.show()
    

In [ ]:
m_Ch_A, ci1_Ch_A, ci2_Ch_A = mean_confidence_interval([x[0] for x in pvals_Ch], confidence=0.95)
m_Ch_B, ci1_Ch_B, ci2_Ch_B = mean_confidence_interval([x[1] for x in pvals_Ch], confidence=0.95)
m_Ch_C, ci1_Ch_C, ci2_Ch_C = mean_confidence_interval([x[2] for x in pvals_Ch], confidence=0.95)
m_Ch_DRB1, ci1_Ch_DRB1, ci2_Ch_DRB1 = mean_confidence_interval([x[3] for x in pvals_Ch], confidence=0.95)
m_Ch_DQB1, ci1_Ch_DQB1, ci2_Ch_DQB1 = mean_confidence_interval([x[4] for x in pvals_Ch], confidence=0.95)
m_Ch_DPB1, ci1_Ch_DPB1, ci2_Ch_DPB1 = mean_confidence_interval([x[5] for x in pvals_Ch], confidence=0.95)

Ch_HW = [m_Ch_A,m_Ch_B,m_Ch_C,m_Ch_DRB1,m_Ch_DQB1,m_Ch_DPB1]

m_Ret_A, ci1_Ret_A, ci2_Ret_A = mean_confidence_interval([x[0] for x in pvals_Ret], confidence=0.95)
m_Ret_B, ci1_Ret_B, ci2_Ret_B = mean_confidence_interval([x[1] for x in pvals_Ret], confidence=0.95)
m_Ret_C, ci1_Ret_C, ci2_Ret_C = mean_confidence_interval([x[2] for x in pvals_Ret], confidence=0.95)
m_Ret_DRB1, ci1_Ret_DRB1, ci2_Ret_DRB1 = mean_confidence_interval([x[3] for x in pvals_Ret], confidence=0.95)
m_Ret_DQB1, ci1_Ret_DQB1, ci2_Ret_DQB1 = mean_confidence_interval([x[4] for x in pvals_Ret], confidence=0.95)
m_Ret_DPB1, ci1_Ret_DPB1, ci2_Ret_DPB1 = mean_confidence_interval([x[5] for x in pvals_Ret], confidence=0.95)

Ret_HW = [m_Ret_A,m_Ret_B,m_Ret_C,m_Ret_DRB1,m_Ret_DQB1,m_Ret_DPB1]

m_Her_A, ci1_Her_A, ci2_Her_A = mean_confidence_interval([x[0] for x in pvals_Her], confidence=0.95)
m_Her_B, ci1_Her_B, ci2_Her_B = mean_confidence_interval([x[1] for x in pvals_Her], confidence=0.95)
m_Her_C, ci1_Her_C, ci2_Her_C = mean_confidence_interval([x[2] for x in pvals_Her], confidence=0.95)
m_Her_DRB1, ci1_Her_DRB1, ci2_Her_DRB1 = mean_confidence_interval([x[3] for x in pvals_Her], confidence=0.95)
m_Her_DQB1, ci1_Her_DQB1, ci2_Her_DQB1 = mean_confidence_interval([x[4] for x in pvals_Her], confidence=0.95)
m_Her_DPB1, ci1_Her_DPB1, ci2_Her_DPB1 = mean_confidence_interval([x[5] for x in pvals_Her], confidence=0.95)

Her_HW = [m_Her_A,m_Her_B,m_Her_C,m_Her_DRB1,m_Her_DQB1,m_Her_DPB1]

m_Las_A, ci1_Las_A, ci2_Las_A = mean_confidence_interval([x[0] for x in pvals_Las], confidence=0.95)
m_Las_B, ci1_Las_B, ci2_Las_B = mean_confidence_interval([x[1] for x in pvals_Las], confidence=0.95)
m_Las_C, ci1_Las_C, ci2_Las_C = mean_confidence_interval([x[2] for x in pvals_Las], confidence=0.95)
m_Las_DRB1, ci1_Las_DRB1, ci2_Las_DRB1 = mean_confidence_interval([x[3] for x in pvals_Las], confidence=0.95)
m_Las_DQB1, ci1_Las_DQB1, ci2_Las_DQB1 = mean_confidence_interval([x[4] for x in pvals_Las], confidence=0.95)
m_Las_DPB1, ci1_Las_DPB1, ci2_Las_DPB1 = mean_confidence_interval([x[5] for x in pvals_Las], confidence=0.95)

Las_HW = [m_Las_A,m_Las_B,m_Las_C,m_Las_DRB1,m_Las_DQB1,m_Las_DPB1]

In [ ]:
prefectures_HW = obtain_pvalue_from_Arlequin_xml(f'/home/manos/Programs/Arlequin/cretan_HLA/prefectures/new_analysis/HW/merged.res/merged_HW_6.xml', 'HW')

prefectures_HW = [Ch_HW, Ret_HW, Her_HW, Las_HW]
# prefectures_HWE_LD_pvalues(prefectures_HW, analysis='HW')
prefectures_HWE_LD_pvalues(prefectures_HW, analysis='HW', log10=True)

#### DOES NOT RUN BECAUSE OF A NUMERIC ERROR PRINTED ON HERAKLION ARLEQUIN XML OUTPUT
#### SO I SAVED THE P-VALUES WITH THE COMMAND ON TERMINAL BELOW:
#### grep '(P = ' all_prefectures_LD_6.xml |gawk -F'P = ' '{print $2}'| gawk -F',' '{print $1}' > prefectures_LD_pvals.txt 

# prefectures_LD = obtain_pvalue_from_Arlequin_xml(f'/home/manos/Programs/Arlequin/cretan_HLA/prefectures/1744/LD/merged.res/merged_LD_6.xml', 'LD')

ch_LD = [1.00000, 0.00000, 0.00000, 0.44722, 0.00000, 0.00000, 0.00020, 0.00000, 0.00000, 0.00000, 1.00000, 1.00000, 0.99971, 0.65537, 0.00050]
ret_LD = [1.00000, 0.86585, 0.00000, 0.97357, 0.29853, 0.87768, 0.00092, 0.00106, 0.00000, 1.00000, 1.00000, 1.00000, 1.00000, 1.00000, 0.99983]
her_LD = [0.08014, 0.00000, 0.00000, 0.00371, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 1.00000, 1.00000, 1.00000, 0.99482, 0.12632]
las_LD = [1.00000, 0.71459, 0.00000, 1.00000, 0.94716, 0.03956, 0.99814, 0.00000, 0.00001, 0.00000, 1.00000, 1.00000, 1.00000, 0.99865, 0.38261]

prefectures_LD = [ch_LD, ret_LD, her_LD, las_LD]

# prefectures_HWE_LD_pvalues(prefectures_LD, analysis='LD')
prefectures_HWE_LD_pvalues(prefectures_LD, analysis='LD', log10=True)

## PYPOP

In [ ]:
def pypop_input(txt_file):
    
    '''
    This function converts Hapl-o-Mat input to PyPop input for HLA loci
    The Hapl-o-Mat input should contain HLA genotypes of the HLA genes
    -A, -B, -C, -DRB1, -DQB1 and -DPB1 in this order only.
    '''
    
    with open(txt_file, 'r') as f:
        f = f.readlines()
        
    f = [x.replace(':','').strip('\n').split('\t')[1:] for x in f][1:]
    new_header = ['a_1', 'a_2', 'b_1', 'b_2', 'c_1', 'c_2', 'drb1_1', 'drb1_2', 'dqb1_1', 'dqb1_2', 'dpb1_1', 'dpb1_2']
    final = [new_header]+f
    
    tmp = f'/home/manos/Programs/PyPopLinux-0.7.0/HLA/{n_Cretans}/'+txt_file.split('.')[0]+'.pop'
    
    if os.path.exists(tmp):
        os.remove(tmp)
    
    with open(tmp,'w') as f:
        wr = csv.writer(f, delimiter='\t')
        wr.writerows(final)

# pypop_input('Crete_75.txt')
# pypop_input('Chania_75.txt')
# pypop_input('Rethymno_75.txt')
# pypop_input('Heraklion_75.txt')
# pypop_input('Lasithi_75.txt')
# pypop_input('Mylopotamos_all.txt')
# pypop_input('Rest_Rethymno_all.txt')

## PCA

In [ ]:
### In order to compare the allele frequencies of the Cretan population with the DKMS minorities we must
### group the alleles with g resolution as the DKMS did. We will estimate first the haplotype frequencies 
### with the Hapl-o-Mat software, because it can perform the g grouping and then, from the haplotype
### frequencies we will calculate the grouped allele frequencies and compare them with the DKMS minorities'.
### Arlequin cannot perform the grouping, so that is why it is not a suitable program for this task, although
### it does not drop the genotypes with at least one missing value. If we do not drop these lines, the 
### estimated allele frequency would be closer to the real frequency of the sample.

### After we run the Hapl-o-Mat with the g grouping without missing data we calculate the allele frequencies

def af_estimation_from_HaploMat_g_group(hfs_dat, hla_gene, num_of_people, alleles=None, counts=None):
    
    '''
    !!!README!!!
    
    This function calculates the allele frequency of the HLA genes A, B, C, DRB1, DQB1 and DPB1 
    if the haplotypes have them. 
    
    Hapl-o-Mat orders the genes in the haplotypes alphabetically.
    
    The haplotypes with 4 genes should have the HLA genes below in this order only:
    A~B~C~DRB1
    
    The haplotypes with 5 genes should have the HLA genes below in this order only:
    A~B~C~DQB1~DRB1
    
    The haplotypes with 6 genes should have the HLA genes below in this order only:
    A~B~C~DPB1~DQB1~DRB1
    
    Other combinations of the HLA genes can not be used as input.    
    
    This function returns the estimated allele frequencies/counts and alleles of the sample from Hapl-o-Mat.
    '''
    
    if hla_gene not in ('A', 'B', 'C', 'DRB1', 'DQB1', 'DPB1'):
        raise Exception("The hla_gene can only be 'A', 'B', 'C', 'DRB1', 'DQB1' or 'DPB1'")
        
    if counts and alleles:
        raise Exception('Only 1 output can come from this function.')
    
    with open(hfs_dat, 'r') as f:
        f = f.readlines()
        
    f = [x.strip('\n').split('\t') for x in f]
    allele_freq = [(x[0].split('~'), float(x[1])) for x in f]
    allele_freq = [([y.strip('g') for y in x[0]],x[1]) for x in allele_freq]
    
    if hla_gene == 'A':
        n = 0        
    elif hla_gene == 'B':
        n = 1        
    elif hla_gene == 'C':
        n = 2        
    
    elif hla_gene == 'DRB1': # Hapl-o-Mat orders the genes in the haplotypes alphabetically.
        if len(allele_freq[0][0]) == 4:     
            n = 3
        elif len(allele_freq[0][0]) == 5:     
            n = 4    
        elif len(allele_freq[0][0]) == 6:
            n = 5
        else:
            raise Exception('The haplotypes should have 4, 5 or 6 genes in a specific order. Read the documentation.')
    
    elif hla_gene == 'DQB1':
        if len(allele_freq[0][0]) == 4:     
            raise Exception('DQB1 should not be in a haplotype with only 4 genes. Read the documentation.')
        elif len(allele_freq[0][0]) == 5:     
            n = 3    
        elif len(allele_freq[0][0]) == 6:
            n = 4
        else:
            raise Exception('The haplotypes should have 4, 5 or 6 genes in a specific order. Read the documentation.')                 
    
    elif hla_gene == 'DPB1':
        if len(allele_freq[0][0]) == 4:     
            raise Exception('DPB1 should not be in a haplotype with only 4 genes. Read the documentation.')
        elif len(allele_freq[0][0]) == 5:     
            raise Exception('DPB1 should not be in a haplotype with only 5 genes. Read the documentation.')    
        elif len(allele_freq[0][0]) == 6:
            n = 3
        else:
            raise Exception('The haplotypes should have 4, 5 or 6 genes in a specific order. Read the documentation.')
    
    HLA_gene = {el:0 for el in [x[0][n] for x in allele_freq]}
    
    for x in allele_freq:
        HLA_gene[x[0][n]] += x[1]
    
    HLA_gene = {k:v for k, v in sorted(HLA_gene.items(), key=lambda item: item[1], reverse=True)}
    
    if alleles:
        return list(HLA_gene.keys())
    
    elif counts:        
        return {k:round(v*num_of_people) for k,v in HLA_gene.items()} 
        # No need to multiply by 2, because of the 2 genotypes on MAC input
    
    else:
        return HLA_gene

In [ ]:
def pca_pref_dkms(chania, rethymno, heraklion, lasithi, aus, bos, cro, fra, ita, net, por, rom, spa, tur, uk, gre, chi, all_alleles=True, af_filter=None):
    
    ita = ita[ita['gene']!='DQB1']
    tur = tur[tur['gene']!='DQB1']
    
    aus_info = dict(zip(aus['allele'].to_list(), aus['af'].to_list()))
    bos_info = dict(zip(bos['allele'].to_list(), bos['af'].to_list()))
    cro_info = dict(zip(cro['allele'].to_list(), cro['af'].to_list()))
    fra_info = dict(zip(fra['allele'].to_list(), fra['af'].to_list()))
    ita_info = dict(zip(ita['allele'].to_list(), ita['af'].to_list()))
    net_info = dict(zip(net['allele'].to_list(), net['af'].to_list()))
    por_info = dict(zip(por['allele'].to_list(), por['af'].to_list()))
    rom_info = dict(zip(rom['allele'].to_list(), rom['af'].to_list()))
    spa_info = dict(zip(spa['allele'].to_list(), spa['af'].to_list()))
    tur_info = dict(zip(tur['allele'].to_list(), tur['af'].to_list()))
    uk_info = dict(zip(uk['allele'].to_list(), uk['af'].to_list()))
    gre_info = dict(zip(gre['allele'].to_list(), gre['af'].to_list()))
    
    chania_alleles = list(chania.keys())
    rethymno_alleles = list(rethymno.keys())
    heraklion_alleles = list(heraklion.keys())
    lasithi_alleles = list(lasithi.keys())
    aus_alleles = aus['allele'].to_list()
    bos_alleles = bos['allele'].to_list()
    cro_alleles = cro['allele'].to_list()
    fra_alleles = fra['allele'].to_list()
    ita_alleles = ita['allele'].to_list()
    net_alleles = net['allele'].to_list()
    por_alleles = por['allele'].to_list()
    rom_alleles = rom['allele'].to_list()
    spa_alleles = spa['allele'].to_list()
    tur_alleles = tur['allele'].to_list()
    uk_alleles = uk['allele'].to_list()
    gre_alleles = gre['allele'].to_list() 
    
    if not chi.empty:
        chi_info = dict(zip(chi['allele'].to_list(), chi['af'].to_list()))
        chi_alleles = chi['allele'].to_list()
        d = [chania_alleles, rethymno_alleles, heraklion_alleles, lasithi_alleles, aus_alleles, bos_alleles, cro_alleles, fra_alleles, ita_alleles, net_alleles, por_alleles, rom_alleles, spa_alleles, tur_alleles, uk_alleles, gre_alleles, chi_alleles]
    else:
        d = [chania_alleles, rethymno_alleles, heraklion_alleles, lasithi_alleles, aus_alleles, bos_alleles, cro_alleles, fra_alleles, ita_alleles, net_alleles, por_alleles, rom_alleles, spa_alleles, tur_alleles, uk_alleles, gre_alleles]
    
    if all_alleles: 
        final_alleles = list(sorted(set(list(itertools.chain.from_iterable(d)))))
    else:
        final_alleles = list(sorted(set(d[0]).intersection(*d)))
        
    total_alleles = len(final_alleles)
    
    dkms_pool = []
    dkms_pool.append(chania);dkms_pool.append(rethymno);dkms_pool.append(heraklion);dkms_pool.append(lasithi)
    dkms_pool.append(aus_info);dkms_pool.append(bos_info);dkms_pool.append(cro_info);dkms_pool.append(fra_info)
    dkms_pool.append(ita_info);dkms_pool.append(net_info);dkms_pool.append(por_info);dkms_pool.append(rom_info)
    dkms_pool.append(spa_info);dkms_pool.append(tur_info);dkms_pool.append(uk_info);dkms_pool.append(gre_info)
    
    markers = {'Chania':'o', 'Rethymno':'o', 'Heraklion':'o', 'Lasithi':'o', 'Austrians':'v', 
               'Bosnians':'>', 'Croatians':'<', 'French':'1', 'Italians':'2', 'Netherlands':'3', 
               'Portuguese':'s', 'Romanians':'D', 'Spaniards':'|', 'Turkish':'*', 'United Kingdom':'_', 
               'Greeks':'+'}
    
    colors = {'Chania':'olive', 'Rethymno':'mediumseagreen', 'Heraklion':'orangered', 'Lasithi':'goldenrod', 
              'Austrians':'blue', 'Bosnians':'green', 'Croatians':'magenta', 'French':'crimson', 
              'Italians':'orange', 'Netherlands':'aqua' , 'Portuguese':'coral', 'Romanians':'black', 
              'Spaniards':'brown', 'Turkish':'coral', 'United Kingdom':'lime', 'Greeks':'blueviolet'}
            
    populations = ['Chania', 'Rethymno', 'Heraklion', 'Lasithi', 'Austrians', 'Bosnians', 'Croatians', 'French', 'Italians', 'Netherlands', 'Portuguese', 'Romanians', 'Spaniards', 'Turkish', 'United Kingdom', 'Greeks']

    if not chi.empty:  
        dkms_pool.append(chi_info)
        table_tmp = np.zeros([17, total_alleles])
        populations.append('Chinese')
        markers['Chinese']='x'
        colors['Chinese']='navy'
    else:
        table_tmp = np.zeros([16, total_alleles])
    
    for x in range(len(dkms_pool)):
        for y in range(total_alleles):
            if final_alleles[y] in dkms_pool[x].keys():
                table_tmp[x][y] += dkms_pool[x][final_alleles[y]]
                
    df1 = pd.DataFrame(data=table_tmp, index=list(range(1, table_tmp.shape[0]+1)), columns=final_alleles)     

    pca = PCA(n_components = 2)
    pca.fit(df1.values)
    pr_data = pca.fit_transform(df1.values)
    pc1, pc2 = pca.explained_variance_ratio_

    df = pd.DataFrame(dict(x=pr_data[:,0], y=pr_data[:,1], label=populations))

    fig, ax = plt.subplots()
    grouped = df.groupby('label')
    
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key], marker=markers[key], s=20)

    plt.xlabel(f'PC1 (variance explained: {round(pc1*100,2)}%)', fontsize = 14)
    plt.ylabel(f'PC2 (variance explained: {round(pc2*100,2)}%)', fontsize = 12)
    plt.gca().invert_xaxis() # So it can be comparable optically with the previous PCA plots with the whole Crete
    
#     if not chi.empty: 
#         if all_alleles:             
#             if af_filter:
#                 plt.title("PCA on Cretan Prefectures and DKMS\n(with Chinese), ALL alleles, FILTERED", fontsize=13)                
#             else:
#                 plt.title("PCA on Cretan Prefectures and DKMS\n(with Chinese), ALL alleles", fontsize=13)
                
#         else:            
#             if af_filter:
#                 plt.title("PCA on Cretan Prefectures and DKMS\n(with Chinese), COMMON alleles, FILTERED", fontsize=12)    
#             else:
#                 plt.title("PCA on Cretan Prefectures and DKMS\n(with Chinese), COMMON alleles", fontsize=13)       
#     else:
#         if all_alleles:             
#             if af_filter:
#                 plt.title("PCA on Cretan Prefectures and\nDKMS, ALL alleles, FILTERED", fontsize=15)           
#             else:
#                 plt.title("PCA on Cretan Prefectures and\nDKMS, ALL alleles", fontsize=15)
                
#         else:            
#             if af_filter:
#                 plt.title("PCA on Cretan Prefectures and\nDKMS, COMMON alleles, FILTERED", fontsize=14)                
#             else:
#                 plt.title("PCA on Cretan Prefectures and\nDKMS, COMMON alleles", fontsize=15)
    
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#     fig.set_dpi(300)
#     fig.tight_layout()
    
#     if not chi.empty: 
#         if all_alleles:             
#             if af_filter:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS with Chinese ALL alleles FILTERED.png")                
#             else:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS with Chinese ALL alleles.png")
                
#         else:            
#             if af_filter:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS with Chinese COMMON alleles FILTERED.png")                
#             else:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS with Chinese COMMON alleles.png")            
#     else:
#         if all_alleles:             
#             if af_filter:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS ALL alleles FILTERED.png")                
#             else:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS ALL alleles.png")
                
#         else:            
#             if af_filter:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS COMMON alleles FILTERED.png")                
#             else:
#                 plt.savefig("Prefectures_Analysis/PCA on Cretan Prefectures and DKMS COMMON alleles.png")
        
    plt.show()

    
    
    
    
#The hfs.dat file is a Haplomat output

ch_4g_path = '/home/manos/Programs/Hapl-o-Mat/test/Chania/4genes_g/hfs.dat'   
ret_4g_path = '/home/manos/Programs/Hapl-o-Mat/test/Rethymno/4genes_g/hfs.dat'
her_4g_path = '/home/manos/Programs/Hapl-o-Mat/test/Heraklion/4genes_g/hfs.dat'
las_4g_path = '/home/manos/Programs/Hapl-o-Mat/test/Lasithi/4genes_g/hfs.dat'

chania_Ag = af_estimation_from_HaploMat_g_group(ch_4g_path, 'A', 455)     
chania_Bg = af_estimation_from_HaploMat_g_group(ch_4g_path, 'B', 455)
chania_Cg = af_estimation_from_HaploMat_g_group(ch_4g_path, 'C', 455)
chania_DRB1g = af_estimation_from_HaploMat_g_group(ch_4g_path, 'DRB1', 455)

rethymno_Ag = af_estimation_from_HaploMat_g_group(ret_4g_path, 'A', 280)     
rethymno_Bg = af_estimation_from_HaploMat_g_group(ret_4g_path, 'B', 280)
rethymno_Cg = af_estimation_from_HaploMat_g_group(ret_4g_path, 'C', 280)
rethymno_DRB1g = af_estimation_from_HaploMat_g_group(ret_4g_path, 'DRB1', 280)

heraklion_Ag = af_estimation_from_HaploMat_g_group(her_4g_path, 'A', 678)     
heraklion_Bg = af_estimation_from_HaploMat_g_group(her_4g_path, 'B', 678)
heraklion_Cg = af_estimation_from_HaploMat_g_group(her_4g_path, 'C', 678)
heraklion_DRB1g = af_estimation_from_HaploMat_g_group(her_4g_path, 'DRB1', 678)

lasithi_Ag = af_estimation_from_HaploMat_g_group(las_4g_path, 'A', 331)     
lasithi_Bg = af_estimation_from_HaploMat_g_group(las_4g_path, 'B', 331)
lasithi_Cg = af_estimation_from_HaploMat_g_group(las_4g_path, 'C', 331)
lasithi_DRB1g = af_estimation_from_HaploMat_g_group(las_4g_path, 'DRB1', 331)

chania_4g = {**chania_Ag, **chania_Bg, **chania_Cg, **chania_DRB1g}
rethymno_4g = {**rethymno_Ag, **rethymno_Bg, **rethymno_Cg, **rethymno_DRB1g}
heraklion_4g = {**heraklion_Ag, **heraklion_Bg, **heraklion_Cg, **heraklion_DRB1g}
lasithi_4g = {**lasithi_Ag, **lasithi_Bg, **lasithi_Cg, **lasithi_DRB1g}

chania_4gb = {k:v for k,v in chania_4g.items() if v > round(1/(2*455), 6)}
rethymno_4gb = {k:v for k,v in rethymno_4g.items() if v > round(1/(2*280), 6)}
heraklion_4gb = {k:v for k,v in heraklion_4g.items() if v > round(1/(2*678), 6)}
lasithi_4gb = {k:v for k,v in lasithi_4g.items() if v > round(1/(2*331), 6)}

# pca_pref_dkms(chania_4g, rethymno_4g, heraklion_4g, lasithi_4g, austrian_minority_in_Germany, bosnian_minority_in_Germany, croatian_minority_in_Germany, french_minority_in_Germany, italian_minority_in_Germany, netherlands_minority_in_Germany, portuguese_minority_in_Germany, romanian_minority_in_Germany, spanish_minority_in_Germany, turkish_minority_in_Germany, united_kingdom_minority_in_Germany, greek_minority_in_Germany, chinese_minority_in_Germany, all_alleles=True)
# pca_pref_dkms(chania_4g, rethymno_4g, heraklion_4g, lasithi_4g, austrian_minority_in_Germany, bosnian_minority_in_Germany, croatian_minority_in_Germany, french_minority_in_Germany, italian_minority_in_Germany, netherlands_minority_in_Germany, portuguese_minority_in_Germany, romanian_minority_in_Germany, spanish_minority_in_Germany, turkish_minority_in_Germany, united_kingdom_minority_in_Germany, greek_minority_in_Germany, pd.DataFrame(), all_alleles=True)
# pca_pref_dkms(chania_4gb, rethymno_4gb, heraklion_4gb, lasithi_4gb, austrian_minority_in_Germany2, bosnian_minority_in_Germany2, croatian_minority_in_Germany2, french_minority_in_Germany2, italian_minority_in_Germany2, netherlands_minority_in_Germany2, portuguese_minority_in_Germany2, romanian_minority_in_Germany2, spanish_minority_in_Germany2, turkish_minority_in_Germany2, united_kingdom_minority_in_Germany2, greek_minority_in_Germany2, chinese_minority_in_Germany2, all_alleles=True, af_filter=True)
# pca_pref_dkms(chania_4gb, rethymno_4gb, heraklion_4gb, lasithi_4gb, austrian_minority_in_Germany2, bosnian_minority_in_Germany2, croatian_minority_in_Germany2, french_minority_in_Germany2, italian_minority_in_Germany2, netherlands_minority_in_Germany2, portuguese_minority_in_Germany2, romanian_minority_in_Germany2, spanish_minority_in_Germany2, turkish_minority_in_Germany2, united_kingdom_minority_in_Germany2, greek_minority_in_Germany2, pd.DataFrame(), all_alleles=True, af_filter=True)
# pca_pref_dkms(chania_4g, rethymno_4g, heraklion_4g, lasithi_4g, austrian_minority_in_Germany, bosnian_minority_in_Germany, croatian_minority_in_Germany, french_minority_in_Germany, italian_minority_in_Germany, netherlands_minority_in_Germany, portuguese_minority_in_Germany, romanian_minority_in_Germany, spanish_minority_in_Germany, turkish_minority_in_Germany, united_kingdom_minority_in_Germany, greek_minority_in_Germany, chinese_minority_in_Germany, all_alleles=False)
# pca_pref_dkms(chania_4g, rethymno_4g, heraklion_4g, lasithi_4g, austrian_minority_in_Germany, bosnian_minority_in_Germany, croatian_minority_in_Germany, french_minority_in_Germany, italian_minority_in_Germany, netherlands_minority_in_Germany, portuguese_minority_in_Germany, romanian_minority_in_Germany, spanish_minority_in_Germany, turkish_minority_in_Germany, united_kingdom_minority_in_Germany, greek_minority_in_Germany, pd.DataFrame(), all_alleles=False)
# pca_pref_dkms(chania_4gb, rethymno_4gb, heraklion_4gb, lasithi_4gb, austrian_minority_in_Germany2, bosnian_minority_in_Germany2, croatian_minority_in_Germany2, french_minority_in_Germany2, italian_minority_in_Germany2, netherlands_minority_in_Germany2, portuguese_minority_in_Germany2, romanian_minority_in_Germany2, spanish_minority_in_Germany2, turkish_minority_in_Germany2, united_kingdom_minority_in_Germany2, greek_minority_in_Germany2, chinese_minority_in_Germany2, all_alleles=False, af_filter=True)
pca_pref_dkms(chania_4gb, rethymno_4gb, heraklion_4gb, lasithi_4gb, austrian_minority_in_Germany2, bosnian_minority_in_Germany2, croatian_minority_in_Germany2, french_minority_in_Germany2, italian_minority_in_Germany2, netherlands_minority_in_Germany2, portuguese_minority_in_Germany2, romanian_minority_in_Germany2, spanish_minority_in_Germany2, turkish_minority_in_Germany2, united_kingdom_minority_in_Germany2, greek_minority_in_Germany2, pd.DataFrame(), all_alleles=False, af_filter=True)

## clustermap

In [ ]:
#cr,aus,bos,cro,fre,gre,ita,net,por,rom,spa,tur,uni: Estimated Haplotypes for each country

def calculate_distances(cr,aus,bos,cro,fre,gre,ita,net,por,rom,spa,tur,uni, euclidean=None, prevosti=None, chi=None):
    
    if sum([x for x in [euclidean,prevosti] if x])==0 or sum([x for x in [euclidean,prevosti] if x])==2:
        raise Exception("Choose either Euclidean or Prevosti's distance.")

    if chi:
        population_list = [cr,aus,bos,chi,cro,fre,gre,ita,net,por,rom,spa,tur,uni]
        population_names = ['Crete', 'Austria', 'Bosnia-Erzegovina', 'China', 'Croatia', 'France', 'Greece', 'Italy', 'Netherlands' ,'Portugal', 'Romania', 'Spain', 'Turkey', 'United kingdom']
                
    else:
        population_list = [cr,aus,bos,cro,fre,gre,ita,net,por,rom,spa,tur,uni]
        population_names = ['Crete', 'Austria', 'Bosnia-Erzegovina', 'Croatia', 'France', 'Greece', 'Italy', 'Netherlands' ,'Portugal', 'Romania', 'Spain', 'Turkey', 'United kingdom']
    
    arr = np.zeros((len(population_names),len(population_names)))
    
    for x in range(len(population_list)):
        for y in range(len(population_names)):
            if population_list[x] != population_list[y]:
                
                all_haplotypes = list(set(list(population_list[x].keys())+list(population_list[y].keys())))
                a = [population_list[x][z] if z in population_list[x].keys() else 0 for z in all_haplotypes]
                b = [population_list[y][z] if z in population_list[y].keys() else 0 for z in all_haplotypes]                
 
                if euclidean:
                    arr[x][y] += np.linalg.norm(np.array(a)-np.array(b))
                    
                elif prevosti: # similar to Manhattan distance
                    arr[x][y] += np.sum(np.abs(np.array(a)-np.array(b)))*0.5
                
    df = pd.DataFrame(data=arr, index=population_names, columns=population_names)            
            
    
    return df

euc_distances = calculate_distances(cr_hapl,austrian_hapl_freq,bosnian_hapl_freq,croatia_hapl_freq,french_hapl_freq,greek_hapl_freq,italian_hapl_freq,netherland_hapl_freq,portugal_hapl_freq,romanian_hapl_freq,spanish_hapl_freq,turkish_hapl_freq,united_kingdom_hapl_freq, euclidean=True, prevosti=None)
prev_distances = calculate_distances(cr_hapl,austrian_hapl_freq,bosnian_hapl_freq,croatia_hapl_freq,french_hapl_freq,greek_hapl_freq,italian_hapl_freq,netherland_hapl_freq,portugal_hapl_freq,romanian_hapl_freq,spanish_hapl_freq,turkish_hapl_freq,united_kingdom_hapl_freq, euclidean=None, prevosti=True)


In [ ]:
# g = sns.clustermap(prev_distances)
g = sns.clustermap(prev_distances, cmap='Greys_r')

plt.tight_layout()
# g.savefig(f"Cluster Map {n_Cretans}.png", dpi=300)